Kaggle link: https://www.kaggle.com/c/expedia-hotel-recommendations

---

**Raw datasets analysis：**

The first step is to look at the feature description of the dataset. We can see that the dataset records the search information of some Expedia users. There are three files in the dataset atmosphere, test.csv, train.csv and destination.csv. Test.csv/train.csv contains aggregated information about which hotels the customer searched and ultimately booked. Destinations.csv includes information about regional users searching for hotels

Data Field

train/test.csv

|Column name|	Description	|Data type
|--------------------------------------------------------------|
|date_time	|Timestamp	|string
|site_name	|ID of the Expedia point of sale (i.e. Expedia.com, Expedia.co.uk, Expedia.co.jp, ...)	|int
|posa_continent	|ID of continent associated with site_name		|int
|user_location_country	|The ID of the country the customer is located	|int
|user_location_region	|The ID of the region the customer is located	|int
|user_location_city	|The ID of the city the customer is located|	int
|orig_destination_distance	|Physical distance between a hotel and a customer at the time of search. A null means the distance could not be calculated	|double
|user_id	|ID of user|	int
|is_mobile	|1 when a user connected from a mobile device, 0 otherwise|	tinyint
|is_package	|1 if the click/booking was generated as a part of a package (i.e. combined with a flight), 0 otherwise	|int
|channel	|ID of a marketing channel	|int
|srch_ci	|Checkin date	|string
|srch_co	|Checkout date	|string
|srch_adults_cnt	|The number of adults specified in the hotel room|	int
|srch_children_cnt	|The number of (extra occupancy) children specified in the hotel room	|int
|srch_rm_cnt	|The number of hotel rooms specified in the search|	int
|srch_destination_id	|ID of the destination where the hotel search was performed	|int
|srch_destination_type_id	|Type of destination	|int
|hotel_continent	|Hotel continent	|int
|hotel_country	|Hotel country	|int
|hotel_market	|Hotel market	|int
|is_booking	|1 if a booking, 0 if a click	|tinyint
|cnt	|Numer of similar events in the context of the same user session	|bigint
|hotel_cluster	|ID of a hotel cluster	|int

---
destinations.csv

|Column name	|Description	|Data type
|----------------------------------------------------------|
|srch_destination_id	|ID of the destination where the hotel search was performed	|int
|d1-d149	|latent description of search regions	|double


Because the competition use the real data that the user generated at Expedia, we can take some time to learn about the Expedia website. Experiencing the booking process can help us better understand Expedia data in the context of data description.

|Website Info|Dataset Column|
|-|
|Going To|srch_destination_type_id, hotel_continent, hotel_country, hotel_market
|Check-in|srch_ci
|Check-out|srch_co
|Guests|srch_adults_cnt, srch_children_cnt, srch_rm_cnt
|Add a Flight|is_package
|Website name，Expedia.com or else |site_name
|User's location, device, session on Expedia website (Website records users' sessions)|user_location_country, user_location_region, user_location_city, is_mobile,channel, is_booking, cnt
||

**Data files：**

|File|Size|Num of entries|Num of Columns
|-|
|train.csv|3.79GB|37670293|24
|test.csv|263MB|2528243|23
|destinations.csv|131MB|62106|150

In [1]:
import pandas as pd

In [2]:
# Load the train dataset
df_train = pd.read_csv("train.csv")
print(df_train.head())
print(df_train.shape)
print(df_train.info())

             date_time  site_name  posa_continent  user_location_country  \
0  2014-08-11 07:46:59          2               3                     66   
1  2014-08-11 08:22:12          2               3                     66   
2  2014-08-11 08:24:33          2               3                     66   
3  2014-08-09 18:05:16          2               3                     66   
4  2014-08-09 18:08:18          2               3                     66   

   user_location_region  user_location_city  orig_destination_distance  \
0                   348               48862                  2234.2641   
1                   348               48862                  2234.2641   
2                   348               48862                  2234.2641   
3                   442               35390                   913.1932   
4                   442               35390                   913.6259   

   user_id  is_mobile  is_package      ...        srch_children_cnt  \
0       12          0      

**By observing train.csv we can find：**

* data_time should be useful for our predictions, so we need to convert it.
* Most of the feature values are integer and floating point, so we can't do much feature engineering. For example, user_location_country is not the name of the country but is an integer value. This makes it difficult to construct new features because we don't know what that value really means.

In [3]:
import pandas as pd
df_test = pd.read_csv("test.csv")
print(df_test.head())
print(df_test.shape)
print(df_test.info())

   id            date_time  site_name  posa_continent  user_location_country  \
0   0  2015-09-03 17:09:54          2               3                     66   
1   1  2015-09-24 17:38:35          2               3                     66   
2   2  2015-06-07 15:53:02          2               3                     66   
3   3  2015-09-14 14:49:10          2               3                     66   
4   4  2015-07-17 09:32:04          2               3                     66   

   user_location_region  user_location_city  orig_destination_distance  \
0                   174               37449                  5539.0567   
1                   174               37449                  5873.2923   
2                   142               17440                  3975.9776   
3                   258               34156                  1508.5975   
4                   467               36345                    66.7913   

   user_id  is_mobile      ...          srch_ci     srch_co srch_adults_cn

**By observing test.csv we can find：**

* The date of test.csv is later than train.csv, and the data page confirms this. The date of the test dataset starts in 2015, and the date of the training dataset is from 2013 to 2014.
* It seems that the user ids of test.csv seems to be a subset of the train.csv user ids, which is a cross-integrated tree range. We will confirm this later.
* The value of the is_booking column in test.csv is always 1, and the data page can confirm this.


In [5]:
# Load the destination dataset
df_dest = pd.read_csv("destinations.csv")
print(df_dest.head())
print(df_dest.shape)

   srch_destination_id        d1        d2        d3        d4        d5  \
0                    0 -2.198657 -2.198657 -2.198657 -2.198657 -2.198657   
1                    1 -2.181690 -2.181690 -2.181690 -2.082564 -2.181690   
2                    2 -2.183490 -2.224164 -2.224164 -2.189562 -2.105819   
3                    3 -2.177409 -2.177409 -2.177409 -2.177409 -2.177409   
4                    4 -2.189562 -2.187783 -2.194008 -2.171153 -2.152303   

         d6        d7        d8        d9    ...         d140      d141  \
0 -1.897627 -2.198657 -2.198657 -1.897627    ...    -2.198657 -2.198657   
1 -2.165028 -2.181690 -2.181690 -2.031597    ...    -2.165028 -2.181690   
2 -2.075407 -2.224164 -2.118483 -2.140393    ...    -2.224164 -2.224164   
3 -2.115485 -2.177409 -2.177409 -2.177409    ...    -2.161081 -2.177409   
4 -2.056618 -2.194008 -2.194008 -2.145911    ...    -2.187356 -2.194008   

       d142      d143      d144      d145      d146      d147      d148  \
0 -2.198657 -2.19

**By observing destinations.csv we can find：**

* Destinations contains an id associated with srch_destination_id and 149 columns of latent information.
* Expedia does not tell us the true meaning of each latent feature, but it can be assumed that some of the components of the purpose and purpose are like names, descriptions, or others. These hidden features will be converted from values, so they will become anonymous

**Hotel clusters：**

We can use the value_counts method to preliminarily calculate the distribution of cluster:

In [6]:
df_train["hotel_cluster"].value_counts()

91    1043720
41     772743
48     754033
64     704734
65     670960
5      620194
98     589178
59     570291
42     551605
21     550092
70     545572
18     545284
83     534132
46     534038
25     530591
62     518809
95     509266
28     507016
68     503797
82     503755
37     496061
50     489892
30     489287
9      488328
58     483253
97     479446
16     477868
72     457463
1      452694
99     444887
       ...   
19     282893
84     278264
66     273505
38     269246
87     260398
23     259233
12     259022
31     257587
67     255946
43     253578
7      252447
54     250745
92     244343
89     243560
45     241408
49     240124
3      225250
80     220218
60     217919
71     216054
93     214293
86     209054
14     192299
75     165226
24     164127
35     139122
53     134812
88     107784
27     105040
74      48355
Name: hotel_cluster, Length: 100, dtype: int64

It can be seen that the number of hotels in each cluster is fairly evenly distributed, which does not show any link between the cluster number and the number.

**Explore user_id in training and test data:**

Finally, let's verify our hypothesis that all user ids in the test dataset can be found in the train dataset.

In [7]:
test_ids = set(df_test.user_id.unique())
train_ids = set(df_train.user_id.unique())
intersection_count = len(test_ids & train_ids)
print(intersection_count) 
print(len(test_ids))

1181577
1181577


This indicates that many users just browsed but did not actually book.

In [8]:
df_train['is_booking'].value_counts()

0    34669600
1     3000693
Name: is_booking, dtype: int64